In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from datetime import datetime
from tqdm import tqdm

def setup_driver():
    """Chrome 드라이버를 설정하고 반환합니다."""
    chrome_driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
    service = Service(chrome_driver_path)

    options = Options()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome(service=service, options=options)
    return driver

def crawl_blog_posts(search_queries, max_posts_per_query=300, max_pages=10):
    current_time = datetime.now().strftime('%y-%m-%d')
    
    driver = setup_driver()

    all_data = []

    for search_query in search_queries:
        search_query_encoded = search_query.replace(' ', '+')  # 검색어를 URL 인코딩
        url_template = f'https://section.blog.naver.com/Search/Post.naver?pageNo={{}}&rangeType=ALL&orderBy=sim&keyword={search_query_encoded}'
        
        url_list = []
        title_list = []
        seen_urls = set()  # 중복 확인을 위한 집합(Set)

        for page in range(1, max_pages + 1):
            search_url = url_template.format(page)
            driver.get(search_url)
            time.sleep(3)
            
            try:
                # 블로그 글 제목과 URL 수집
                article_raw = driver.find_elements(By.CSS_SELECTOR, '.info_post .desc a')
                
                if not article_raw:
                    print(f"페이지 {page}에서 블로그 포스트를 찾을 수 없습니다.")
                    break
                
                for article in article_raw:
                    url = article.get_attribute('href')
                    
                    # 이미 수집된 URL인지 확인 (중복 제거)
                    if url in seen_urls:
                        continue  # 중복된 URL은 무시
                    
                    seen_urls.add(url)  # 새로운 URL은 집합에 추가
                    
                    title_element = article.find_element(By.XPATH, './strong')
                    title = title_element.text
                    url_list.append(url)
                    title_list.append(title)

                    if len(url_list) >= max_posts_per_query:
                        break
                
                if len(url_list) >= max_posts_per_query:
                    break
            except Exception as e:
                print(f"페이지 {page}에서 오류 발생: {e}")
                continue
        
        # 수집한 URL과 제목을 DataFrame에 저장
        df = pd.DataFrame({'주소': url_list, '제목': title_list})
        
        # 크롤링한 내용을 포함한 DataFrame을 Excel 파일로 저장
        df['검색어'] = search_query
        df['작성일자'] = None
        df['태그'] = None
        df['내용'] = None

        number = len(df)

        for i in tqdm(range(number), desc=f"Crawling: {search_query}"):
            url = df['주소'][i]
            driver.get(url)    
            time.sleep(3)

            try:
                # 블로그 본문 iframe으로 전환
                WebDriverWait(driver, 10).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, 'mainFrame'))
                )

                # 제목 추출
                tit = driver.find_element(By.CSS_SELECTOR, '.se-module.se-module-text.se-title-text')
                title = tit.text

                # 날짜 및 시간 추출
                date = driver.find_element(By.CSS_SELECTOR, '.se_publishDate.pcol2')
                datetime_text = date.text

                df.at[i, '작성일자'] = datetime_text

                # 태그 추출
                tags = driver.find_elements(By.CSS_SELECTOR, '.wrap_tag .ell')
                tag_list = [tag.text for tag in tags]
                tag_str = ' '.join(tag_list)
                df.at[i, '태그'] = tag_str

                # 내용 추출
                contents = driver.find_elements(By.CSS_SELECTOR, '.se-module.se-module-text')
                content_list = [content.text for content in contents]
                content_str = ' '.join(content_list)
                df.at[i, '내용'] = content_str

                print(f"{i}번째 글 크롤링 완료: {title}")

            except Exception as e:
                print(f"{i}번째 글에서 오류 발생: {e}")
                continue

        all_data.append(df)

    all_df = pd.concat(all_data, ignore_index=True)
    all_df.to_excel(f"네이버블로그크롤링_{current_time}.xlsx", index=False, encoding = 'UTF-8')

    print("수집한 글 갯수:", len(all_df.dropna(subset=['내용'])))

    driver.quit()

# 함수 호출 예제
search_queries = [
    # 발달상권
    "성수카페거리", "가로수길", "북촌한옥마을", "인사동·익선동", "광장(전통)시장", "압구정로데오거리",
    
    # 인구밀집지역
    "홍대입구역(2호선)", "이태원역", "강남역", "서울식물원·마곡나루역",
    
    # 관광특구
    "동대문 관광특구", "명동 관광특구", "잠실 관광특구", "종로·청계 관광특구",
    
    # 고궁 문화유산
    "경복궁", "광화문·덕수궁", "창덕궁·종묘", "보신각",
    
    # 공원
    "반포한강공원", "고척돔", "남산공원", "잠실종합운동장",
    "서울숲공원", "국립중앙박물관·용산가족공원", "서울대공원"
]
crawl_blog_posts(search_queries, max_posts_per_query=100, max_pages=20)

Crawling: 성수카페거리:   1%|▌                                                         | 1/100 [00:04<07:11,  4.36s/it]

0번째 글 크롤링 완료: 성수역 가볼만한곳 니커버커 베이글 성수 카페 거리 추천


Crawling: 성수카페거리:   2%|█▏                                                        | 2/100 [00:08<06:38,  4.06s/it]

1번째 글 크롤링 완료: 서울숲카페거리 서울앵무새 성수 주말서울나들이


Crawling: 성수카페거리:   3%|█▋                                                        | 3/100 [00:11<06:21,  3.93s/it]

...

Crawling: 성수카페거리:   5%|██▉                                                       | 5/100 [00:19<06:14,  3.94s/it]

4번째 글 크롤링 완료: 성수 팝업스토어 N2 나이트 NH투자증권 팝업 성수동 카페거리 이색 데이트


Crawling: 성수카페거리:   6%|███▍                                                      | 6/100 [00:23<06:06,  3.90s/it]

5번째 글에서 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7867E9632+30946]
	(No symbol) [0x00007FF78679E3C9]
	(No symbol) [0x00007FF786696FDA]
	(No symbol) [0x00007FF78666CB85]
	(No symbol) [0x00007FF7867137A7]
	(No symbol) [0x00007FF78672A771]
	(No symbol) [0x00007FF78670C813]
	(No symbol) [0x00007FF7866DA6E5]
	(No symbol) [0x00007FF7866DB021]
	GetHandleVerifier [0x00007FF78691F83D+1301229]
	GetHandleVerifier [0x00007FF78692BDB7+1351783]
	GetHandleVerifier [0x00007FF786922A03+1313971]
	GetHandleVerifier [0x00007FF78681DD06+245686]
	(No symbol) [0x00007FF7867A758F]
	(No symbol) [0x00007FF7867A3804]
	(No symbol) [0x00007FF7867A3992]
	(No symbol) [0x00007FF78679A3EF]
	BaseThreadInitThunk [0x00007FF831517374+20]
	RtlUserThreadStart [0x00007FF831B5CC91+33]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7867E9632+30946]
	(No symbol) [0x00007FF78679E3C9]
	(No symbol) [0x00007FF786696FDA]
	(No symbol) [0x00007FF78666CB85]
	(No symbol) [0x00007FF7867137A7]
	(No symbol) [0x00007FF78672A771]
	(No symbol) [0x00007FF78670C813]
	(No symbol) [0x00007FF7866DA6E5]
	(No symbol) [0x00007FF7866DB021]
	GetHandleVerifier [0x00007FF78691F83D+1301229]
	GetHandleVerifier [0x00007FF78692BDB7+1351783]
	GetHandleVerifier [0x00007FF786922A03+1313971]
	GetHandleVerifier [0x00007FF78681DD06+245686]
	(No symbol) [0x00007FF7867A758F]
	(No symbol) [0x00007FF7867A3804]
	(No symbol) [0x00007FF7867A3992]
	(No symbol) [0x00007FF78679A3EF]
	BaseThreadInitThunk [0x00007FF831517374+20]
	RtlUserThreadStart [0x00007FF831B5CC91+33]


In [2]:
df = pd.read_excel('../data/네이버블로그크롤링_24-08-27.xlsx')
df

,주소,제목,검색어,작성일자,태그,내용
0,https://blog.naver.com/daisy1505/223523300415,성수역 가볼만한곳 니커버커 베이글 성수 카페 거리 추천,성수카페거리,2024. 7. 24. 0:36,#성수가볼만한곳 #성수역가볼만한곳 #니커버커베이글성수점 #성수동카페거리 #성수카페추...,성수역 가볼만한곳 니커버커 베이글 성수 카페 거리 추천 성수역 가볼만한곳\n니커버커...
1,https://blog.naver.com/buool7/223556412875,서울숲카페거리 서울앵무새 성수 주말서울나들이,성수카페거리,2024. 8. 22. 7:36,#서울숲카페 #서울숲카페거리 #서울앵무새 #서울앵무새성수 #주말나들이 #주말서울나들...,서울숲카페거리 서울앵무새 성수 주말서울나들이 임신막달이 되어가니 자꾸 어딘가\n돌아...
2,https://blog.naver.com/cutetnwls/223545635964,"성수동 카페거리 , 성수 핫플 아미들의 성지 파벤(CAFE FAR BEN) 솔직 후기",성수카페거리,2024. 8. 22. 0:00,NaN,"성수동 카페거리 , 성수 핫플 아미들의 성지 파벤(CAFE FAR BEN) 솔직 후..."
3,https://blog.naver.com/young82k/223486856752,성수역 와인바 분위기 좋은 성수카페거리 맛집으로 추천,성수카페거리,2024. 6. 22. 9:00,#성수역와인바,성수역 와인바 분위기 좋은 성수카페거리 맛집으로 추천 성수역 와인바 분위기 좋은 성...
4,https://blog.naver.com/sanaella/223199341514,"서울앵무새 성수 핫플 솔직평, 성수동카페거리",성수카페거리,2023. 9. 29. 7:50,#서울앵무새 #성수핫플 #서울앵무새성수 #성수동카페거리,"서울앵무새 성수 핫플 솔직평, 성수동카페거리 드디어 다녀온 서울앵무새 성수\n지난번..."
...,...,...,...,...,...,...
2495,https://blog.naver.com/finezoos/223513662613,EVENT 도전 명탐정! 사진 속 서울대공원 장소를 맞춰봐,서울대공원,2024. 7. 16. 11:00,#서울대공원 #seoulgrandpark #이벤트 #EVENT #커피이벤트 #배민이...,NaN
2496,https://blog.naver.com/finezoos/223447725737,서울대공원 앱 다운로드 및 스탬프 이벤트 함께 참여해요,서울대공원,2024. 5. 15. 21:01,#서울대공원 #seoulgrandpark #서울대공원이벤트 #이벤트 #앱다운로드이벤...,NaN
2497,https://blog.naver.com/globalhope/223523975101,[이태원] TWOFMCIS 서울대공원 동물원 소풍,서울대공원,2024. 7. 24. 15:41,#글로벌호프 #globalhope #KOICA #NGO #ODA #국제개발협력 #비...,NaN
2498,https://blog.naver.com/thalia/223473590762,서울대공원 산림욕장길,서울대공원,2024. 6. 16. 15:20,#과천맛집 #서울대공원산림욕장 #서울근교숲길,NaN
